# Variáveis, imports e funções primárias

In [1]:
import struct
import wave
import datetime
import os
import glob
import csv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import re
from datetime import datetime as dt

In [2]:
PACKET_SIZE = 4
ID_SIZE = 4
TIMESTAMP_SIZE = 8
AUDIO_SAMPLING_FREQ = 16384
AUDIO_SAMPLE_SIZE = 3
AUDIO_SIZE = AUDIO_SAMPLE_SIZE * AUDIO_SAMPLING_FREQ
BME280_SIZE = 4 * 4
MECHANICAL_SIZE = 3 * 4

In [3]:
CHECKSUM = True # checksum has been implemented from 2023-08-09 at 15:00
SAMPLE_SIZE = PACKET_SIZE + ID_SIZE + TIMESTAMP_SIZE + AUDIO_SIZE + BME280_SIZE + MECHANICAL_SIZE
if CHECKSUM:
    SAMPLE_SIZE+=1

In [4]:
# Define paths for saving data
save_in_directory = "./sem43Data"
os.makedirs(save_in_directory, exist_ok=True)
csv_filename = f"{save_in_directory}/sem43.csv"

In [6]:
"""
*  \brief     calibration.py
*  \details   Python script to get wind direction from raw data
*  \author    Mateo Haro
*  \version   1.1
*  \date      2022-08-21
*  \pre       None
*  \copyright (c) 2022 CSEM
*
*   CSEM S.A.
*   Jaquet-Droz 1
*   CH-2000 Neuchâtel
*   http://www.csem.ch
*
*
*   THIS PROGRAM IS CONFIDENTIAL AND CANNOT BE DISTRIBUTED
*   WITHOUT THE CSEM PRIOR WRITTEN AGREEMENT.
*
*   CSEM is the owner of this source code and is authorised to use, to modify
*   and to keep confidential all new modifications of this code.
*
"""

WMK_NUM_ANGLES = 16
SFE_WIND_VANE_DEGREES_PER_INDEX = 360/WMK_NUM_ANGLES

class Station:
   def __init__(self, id, calibration):
      self.id = id
      self.vaneADCValues = calibration

   def getWindDirection(self, rawADC):
      """
      We'll loop through all possible directions to find which is closest
      to our measurement, using a simple linear search. closestDifference is
      initialized to max 16-bit signed value (2^15 - 1 = 32,767)
      """
      closestDifference = 32767
      closestIndex = 0

      for i in range(WMK_NUM_ANGLES):
         # Compute the difference between the ADC value for this direction and what we measured
         adcDifference = self.vaneADCValues[i] - rawADC

         # We only care about the magnitude of the difference
         adcDifference = abs(adcDifference)

         # Check if this different is less than our closest so far
         if adcDifference < closestDifference:
            # This resistance is closer, update closest resistance and index
            closestDifference = adcDifference
            closestIndex = i

      # Now compute the wind direction in degrees
      direction = closestIndex * SFE_WIND_VANE_DEGREES_PER_INDEX

      # Return direction in degrees
      # As an indication: 0deg is North and 90deg is East
      return direction

# calibration for mechanical station associated to CSEMAurora1
calibration1 = [2955, 1437, 1658, 172, 208, 105, 568, 336, 965, 797, 2320, 2205, 3876, 3154, 3477, 2609]
station1 = Station(1, calibration1)

# calibration for mechanical station associated to CSEMAurora2
calibration2 = [3050, 1490, 1720, 185, 225, 113, 594, 355, 1007, 831, 2405, 2282, 3930, 3240, 3550, 2700]
station2 = Station(2, calibration2)

# calibration for mechanical station associated to CSEMAurora3
calibration3 = [3012, 1481, 1703, 202, 240, 131, 602, 370, 1008, 835, 2381, 2256, 3865, 3201, 3500, 2670]
station3 = Station(3, calibration2)

# calibration for mechanical station associated to CSEMAurora4
calibration4 = [3018, 1472, 1702, 185, 222, 112, 590, 351, 993, 821, 2375, 2256, 3935, 3217, 3543, 2672]
station3 = Station(4, calibration2)

# calibration for mechanical station associated to CSEMAurora5
calibration5 = [2927, 1423, 1642, 158, 192, 86, 555, 321, 945, 778, 2300, 2181, 3822, 3119, 3434, 2587]
station3 = Station(5, calibration2)


<>:1: SyntaxWarning: invalid escape sequence '\d'
<>:1: SyntaxWarning: invalid escape sequence '\d'
C:\Users\bruno\AppData\Local\Temp\ipykernel_42860\1088622479.py:1: SyntaxWarning: invalid escape sequence '\d'
  """


In [7]:
#limites para os labels
#Chuva fraca: menos de 1 mm/h.
#Chuva moderada: de 2,5 até 10 mm/h.
#Chuva forte: a partir de 10mm
rain_limits = [0.5, 2, 4] #mm/hr

In [8]:
from datetime import datetime as dt
# Function to convert filename to datetime object for sorting
def filename_to_datetime(filename):
    match = re.search(r'(\d{4}-\d{2}-\d{2})_(\d+)', filename)
    if match:
        date_str, hour_str = match.groups()
        return dt.strptime(f'{date_str} {hour_str}', '%Y-%m-%d %H')
    else:
        return dt.min  # return minimum datetime if no match



# Geração do csv

In [ ]:

#example multiple files
def read_data_multiple_files_hour(directory, save_in_directory,csv_filename):
    total_temperature = 0
    total_pressure = 0
    total_altitude = 0
    total_humidity = 0
    total_wind_count = 0
    total_rain_count = 0
    total_wind_direction = 0
    contagemAmostras = 0
    pulses = []
    #sound_data = []
    # Create directory if it doesn't exist
    os.makedirs(save_in_directory, exist_ok=True)

    # Setup for CSV file
    csv_headers = ['timestamp', 'temperature', 'pressure', 'altitude', 'humidity', 'wind_count', 'wind_km_h','rain_count', 'rain_mm', 'wind_direction', 'audio_file', 'label', 'amostras','pulses']

    # We will check if the CSV already exists to decide whether to write headers or not
    write_headers = not os.path.exists(csv_filename)

    with open(csv_filename, 'a', newline='') as csv_file:
        csv_writer = csv.DictWriter(csv_file, fieldnames=csv_headers)
        if write_headers:
            csv_writer.writeheader()

        # Get a list of all .dat files in the directory
        dat_files = glob.glob(os.path.join(directory, '*.dat'))

        # Sort the files by filename
        dat_files.sort(key=filename_to_datetime)

        for file_name in dat_files:
            try: 
                print(f"Processing {file_name}...")
                # open the .dat file with "rb" (read binary) mode
                with open(file_name, "rb") as f:
                    data = f.read()
                    f.close()
                
                #o timestamp deve dever o nome do arquivo raw_data_CSEMAurora1_2023-08-28_0 alterado para o formato 2023-08-27 00_00_00
                timestamp_name = file_name.split('_')[3] + ' ' + file_name.split('_')[4][:-4] + '_' + '00' + '_' + '00'

                # read the binary data
                num_sample = int(len(data) / SAMPLE_SIZE) # should be an integer
                for j in range(num_sample):
                    index = j * SAMPLE_SIZE
                    packet_size = struct.unpack("<L", data[index: index + PACKET_SIZE])[0] # should be equal to SAMPLE_SIZE
                    index += PACKET_SIZE
                    packet_id = struct.unpack("<L", data[index: index + ID_SIZE])[0]
                    index += ID_SIZE
                    timestamp = struct.unpack("<Q", data[index: index + TIMESTAMP_SIZE])[0]
                    index += TIMESTAMP_SIZE
                    samples_raw =  data[index: index + AUDIO_SIZE]
                    index += AUDIO_SIZE
                    bme_values = list(struct.iter_unpack("<f", data[index: index + BME280_SIZE]))
                    index += BME280_SIZE
                    mechanical_values = list(struct.iter_unpack("<I", data[index: index + MECHANICAL_SIZE]))
                    index += MECHANICAL_SIZE

                    # post process and save the BME and mechanical data in a dict
                    BME_data = {"temperature":bme_values[0][0], "pressure":bme_values[1][0], "altitude":bme_values[2][0], "humidity":bme_values[3][0]}
                    mechanical_data = {"wind_count":mechanical_values[0][0],"rain_count":mechanical_values[1][0],"wind_direction":mechanical_values[2][0]}

                    if mechanical_data['rain_count'] > 0:
                        pulses.append(j)

                    total_temperature += BME_data['temperature']
                    total_pressure += BME_data['pressure']
                    total_altitude += BME_data['altitude']
                    total_humidity += BME_data['humidity']
                    total_wind_count += mechanical_data['wind_count']
                    total_rain_count += mechanical_data['rain_count']
                    total_wind_direction += mechanical_data['wind_direction']
                    contagemAmostras += 1
                
                med_temperature = total_temperature / contagemAmostras
                med_pressure = total_pressure / contagemAmostras
                med_altitude = total_altitude / contagemAmostras
                med_humidity = total_humidity / contagemAmostras
                med_wind_count = total_wind_count / contagemAmostras
                med_wind_direction = total_wind_direction / contagemAmostras
                wind_km_h = med_wind_count * 2.4
                rain_mm = total_rain_count * 0.2794
                wind_deg = station1.getWindDirection(med_wind_direction)
                label = ''
                if rain_mm < rain_limits[0]:
                    label = 0
                elif rain_mm < rain_limits[1]:
                    label = 1
                elif rain_mm < rain_limits[2]:
                    label = 2
                else:
                    label = 3

                row_data = {
                    'timestamp': timestamp_name,
                    'temperature': med_temperature,
                    'pressure': med_pressure,
                    'altitude': med_altitude,
                    'humidity': med_humidity,
                    'wind_count': med_wind_count,
                    'wind_km_h': wind_km_h,
                    'rain_count': total_rain_count,
                    'rain_mm': rain_mm,
                    'wind_direction': wind_deg,
                    'audio_file': 'null',
                    'label': label,
                    'amostras': contagemAmostras,
                    'pulses': pulses
                }
                csv_writer.writerow(row_data)

                total_temperature = 0
                total_pressure = 0
                total_altitude = 0
                total_humidity = 0
                total_wind_count = 0
                total_rain_count = 0
                total_wind_direction = 0
                contagemAmostras = 0
                pulses = []
                #sound_data = []
                time_ini = ''
                time_ini_name =''
            except:
                
                print(f"Error processing {file_name}...")


In [14]:
read_data_multiple_files_hour('./rawdata43', save_in_directory, csv_filename)

Processing ./rawdata43\raw_data_CSEMAurora1_2023-10-26_0.dat...
Processing ./rawdata43\raw_data_CSEMAurora1_2023-10-26_1.dat...
Processing ./rawdata43\raw_data_CSEMAurora1_2023-10-26_2.dat...
Processing ./rawdata43\raw_data_CSEMAurora1_2023-10-26_3.dat...
Processing ./rawdata43\raw_data_CSEMAurora1_2023-10-26_4.dat...
Processing ./rawdata43\raw_data_CSEMAurora1_2023-10-26_5.dat...
Processing ./rawdata43\raw_data_CSEMAurora1_2023-10-26_6.dat...
Processing ./rawdata43\raw_data_CSEMAurora1_2023-10-26_7.dat...
Processing ./rawdata43\raw_data_CSEMAurora1_2023-10-26_8.dat...
Processing ./rawdata43\raw_data_CSEMAurora1_2023-10-26_9.dat...
Processing ./rawdata43\raw_data_CSEMAurora1_2023-10-26_10.dat...
Processing ./rawdata43\raw_data_CSEMAurora1_2023-10-26_11.dat...
Processing ./rawdata43\raw_data_CSEMAurora1_2023-10-26_12.dat...
Processing ./rawdata43\raw_data_CSEMAurora1_2023-10-26_13.dat...
Processing ./rawdata43\raw_data_CSEMAurora1_2023-10-26_14.dat...
Processing ./rawdata43\raw_data_CSE

In [13]:
df = pd.read_csv('sem43Data/sem43.csv') 
#mudar a clouna do csv para ficar no formato 2023-08-28_0, onde o ultimo numero é a hora
df['timestamp'] = df['timestamp'].apply(lambda x: x.split('_')[0])
#subtituir espaço vaizo por _
df['timestamp'] = df['timestamp'].apply(lambda x: x.replace(' ', '_'))
df.head()
#salvar esse df em um csv
df.to_csv('sem43Data\sem43.csv', index=False)

#contar quantas linhas tem para cada label
df['label'].value_counts()
df['timestamp'].values

<>:8: SyntaxWarning: invalid escape sequence '\s'
<>:8: SyntaxWarning: invalid escape sequence '\s'
C:\Users\bruno\AppData\Local\Temp\ipykernel_42860\600523390.py:8: SyntaxWarning: invalid escape sequence '\s'
  df.to_csv('sem43Data\sem43.csv', index=False)


array(['2023-10-26', '2023-10-26', '2023-10-26', '2023-10-26',
       '2023-10-26', '2023-10-26', '2023-10-27', '2023-10-27',
       '2023-10-26_0', '2023-10-26_1', '2023-10-26_2', '2023-10-26_3',
       '2023-10-26_4', '2023-10-26_5', '2023-10-26_6', '2023-10-26_7',
       '2023-10-26_8', '2023-10-26_9', '2023-10-26_10', '2023-10-26_11',
       '2023-10-26_12', '2023-10-26_13', '2023-10-26_14', '2023-10-26_15',
       '2023-10-26_16', '2023-10-26_17', '2023-10-26_18', '2023-10-26_19',
       '2023-10-26_20', '2023-10-26_21', '2023-10-26_22', '2023-10-26_23',
       '2023-10-27_0', '2023-10-27_1', '2023-10-27_2', '2023-10-27_3',
       '2023-10-27_4', '2023-10-27_5', '2023-10-27_6', '2023-10-27_7',
       '2023-10-27_8', '2023-10-27_9', '2023-10-27_10', '2023-10-27_11',
       '2023-10-27_12', '2023-10-27_13', '2023-10-27_14', '2023-10-27_15',
       '2023-10-27_16', '2023-10-27_17', '2023-10-27_18', '2023-10-27_19',
       '2023-10-27_20', '2023-10-27_21', '2023-10-27_22', '2023-10-27

# Audio

In [15]:
df = pd.read_csv('sem43Data\sem43.csv') 

<>:1: SyntaxWarning: invalid escape sequence '\s'
<>:1: SyntaxWarning: invalid escape sequence '\s'
C:\Users\bruno\AppData\Local\Temp\ipykernel_42860\2519725551.py:1: SyntaxWarning: invalid escape sequence '\s'
  df = pd.read_csv('sem43Data\sem43.csv')


In [ ]:

#example multiple files
def save_audio_data_multiple_files_hour(directory, save_in_directory):

    countLowRain = 20
    countNoRain = 0
    countMediumRain = 99
    countHighRain = 99
    sound_data = []
    label_name = ''
    
    # Create directory if it doesn't exist
    os.makedirs(save_in_directory, exist_ok=True)

    #create directories for each label
    os.makedirs(f"{save_in_directory}/NoRain", exist_ok=True)
    os.makedirs(f"{save_in_directory}/LowRain", exist_ok=True)
    os.makedirs(f"{save_in_directory}/MediumRain", exist_ok=True)
    os.makedirs(f"{save_in_directory}/HighRain", exist_ok=True)

    # Get a list of all .dat files in the directory
    dat_files = glob.glob(os.path.join(directory, '*.dat'))

    # Sort the files by filename
    dat_files.sort(key=filename_to_datetime)
   
    for file_name in dat_files:
        #try: 
            print(f"Processing {file_name}...")         
            
            #o timestamp deve dever o nome do arquivo raw_data_CSEMAurora1_2023-08-28_0 alterado para o formato 2023-08-27 00_00_00
            #ele será comparado com o a coluna timestamp do csv, dependendo do label, o arquivo de audio será salvo em uma pasta específica
            timestamp_name = file_name.split('_')[3] + '_' + file_name.split('_')[4][:-4]
            print(timestamp_name)
            if timestamp_name in df['timestamp'].values:
                print(f"Processing {timestamp_name}...")
                label = df.loc[df['timestamp'] == timestamp_name, 'label'].values[0]
                print(f"Label: {label}")

            #if the four counts are zero, break the loop
            if countNoRain == 0 and countLowRain == 0 and countMediumRain == 0 and countHighRain == 0:
                break

            match label:
                case 0:
                    if countNoRain == 0:
                        continue
                    countNoRain -= 1
                    label_name = 'NoRain'
                case 1:
                    if countLowRain == 0:
                        continue
                    countLowRain -= 1
                    label_name = 'LowRain'
                case 2:
                    if countMediumRain == 0:
                        continue
                    countMediumRain -= 1
                    label_name = 'MediumRain'
                case 3:
                    if countHighRain == 0:
                        continue
                    countHighRain -= 1
                    label_name = 'HighRain'
            
            # open the .dat file with "rb" (read binary) mode
            with open(file_name, "rb") as f:
                data = f.read()
                f.close()            

            # read the binary data
            for pulses in df.loc[df['timestamp'] == timestamp_name, 'pulses']:
                #convertendo a string para lista e tirando espacos vazios
                pulses = pulses[1:-1].split(',')
                
                for pulse in pulses:
                    sound_data = []
                    pulse = int(pulse)
                    start_index = max(0, pulse - 15)
                    end_index = min(len(data), pulse + 15)
                    for j in range(start_index, end_index):
                        index = j * SAMPLE_SIZE
                        index += PACKET_SIZE
                        # packet_id = struct.unpack("<L", data[index: index + ID_SIZE])[0]
                        index += ID_SIZE
                        # timestamp = struct.unpack("<Q", data[index: index + TIMESTAMP_SIZE])[0]
                        index += TIMESTAMP_SIZE
                        samples_raw =  data[index: index + AUDIO_SIZE]
                        index += AUDIO_SIZE
                        index += BME280_SIZE
                        index += MECHANICAL_SIZE

                        # post process the audio samples collected with I2S protocol
                        samples = []
                        for i in range(0, int(AUDIO_SIZE / AUDIO_SAMPLE_SIZE)): # get the number of audio samples, not the number of bytes
                            value =     samples_raw[AUDIO_SAMPLE_SIZE*i      ] << 16
                            value +=    samples_raw[AUDIO_SAMPLE_SIZE*i + 1  ] << 8
                            value +=    samples_raw[AUDIO_SAMPLE_SIZE*i + 2  ]
                            if(value>2**23 - 1):
                                value = -(2**24 - value)
                            samples.append(value)        
                        sound_data += samples
                    
                                    
                    #save sound data in a .wav file
                    sound_file = f"{save_in_directory}/{label_name}/{timestamp_name}_{pulse}.wav"
                    obj = wave.open(sound_file, "w")
                    obj.setnchannels(1)
                    obj.setsampwidth(AUDIO_SAMPLE_SIZE)
                    obj.setframerate(AUDIO_SAMPLING_FREQ)
                    data_as_bytes = [struct.pack("<i", sample) for sample in sound_data] # store it as 4 bytes in LSB format (little endian)
                    for data_bytes in data_as_bytes:
                        obj.writeframes(data_bytes[0:3]) # remove the fourth unused byte one ().
                    obj.close()

In [11]:
#executar a função
directory = './test43'
save_in_directory = './testaudios43'
save_audio_data_multiple_files_hour(directory, save_in_directory)

Processing ./test43\raw_data_CSEMAurora1_2023-10-26_18.dat...
2023-10-26_18
Processing 2023-10-26_18...
Label: 1
Processing ./test43\raw_data_CSEMAurora1_2023-10-26_19.dat...
2023-10-26_19
Processing 2023-10-26_19...
Label: 1


KeyboardInterrupt: 

In [4]:
def split_audio(audio_file, save_in_directory):
    # Create directory if it doesn't exist
    os.makedirs(save_in_directory, exist_ok=True)

    # Open the audio file
    obj = wave.open(audio_file, "r")
    num_frames = obj.getnframes()
    frame_rate = obj.getframerate()
    num_channels = obj.getnchannels()
    sample_width = obj.getsampwidth()
    duration = num_frames / frame_rate
    num_chunks = int(duration / 30) # 30 seconds chunks
    print(f"Duration: {duration} seconds")
    print(f"Number of chunks: {num_chunks}")

    # Read the audio data
    audio_data = obj.readframes(num_frames)
    obj.close()

    # Split the audio data into 30 seconds chunks
    for i in range(num_chunks):
        chunk_file = f"{save_in_directory}/{os.path.basename(audio_file).split('.')[0]}_{i}.wav"
        obj = wave.open(chunk_file, "w")
        obj.setnchannels(num_channels)
        obj.setsampwidth(sample_width)
        obj.setframerate(frame_rate)
        start_frame = int(i * 30 * frame_rate * num_channels * sample_width)
        end_frame = int((i + 1) * 30 * frame_rate * num_channels * sample_width)
        obj.writeframes(audio_data[start_frame:end_frame])
        obj.close()

    # Handle the last chunk if there's any leftover
    if duration % 30 != 0:
        chunk_file = f"{save_in_directory}/{os.path.basename(audio_file).split('.')[0]}_{num_chunks}.wav"
        obj = wave.open(chunk_file, "w")
        obj.setnchannels(num_channels)
        obj.setsampwidth(sample_width)
        obj.setframerate(frame_rate)
        start_frame = int(num_chunks * 30 * frame_rate * num_channels * sample_width)
        obj.writeframes(audio_data[start_frame:])
        obj.close()


#executar a função
audio_file = 'testeAudios/test2.wav'
save_in_directory = './testeAudios/label2'
split_audio(audio_file, save_in_directory)




Duration: 3600.0 seconds
Number of chunks: 120
